In [1]:
from othello_rules import *
from othello_net import *
from tensorflow.python.framework import ops
from datetime import datetime
from example_states import *
from feature_extractor import *
from training_utils import *
from policy_networks import *
import numpy as np
np.set_printoptions(precision=2)

In [2]:
# Ræsum graphið fyrir tensorflow
ops.reset_default_graph()
filters = 32
graph_1, img_data_1, train_step_1, optimizer_1, ground_truths_1, \
                loss_1, pred_up_1, keep_prob_1, learn_rate_1, score_out_1 = policy_net_3x32(filters)
sess_1    = tf.Session(graph=graph_1)
saver_1   = tf.train.Saver()
init_op_1 = tf.initialize_all_variables()
sess_1.run(init_op_1)
model_1 = "models/filters3x32.ckpt"
if os.path.isfile(model_1):
    saver_1.restore(sess_1, model_1)
    
tf.reset_default_graph()

graph_2, img_data_2, train_step_2, optimizer_2, ground_truths_2, \
                loss_2, pred_up_2, keep_prob_2, learn_rate_2, score_out_2 = create_othello_net()
sess_2    = tf.Session(graph=graph_2)
saver_2   = tf.train.Saver()
init_op_2 = tf.initialize_all_variables()
sess_2.run(init_op_2)
model_2 = "models/tiny-rl-t.ckpt"
if os.path.isfile(model_2):
    saver_2.restore(sess_2, model_2)
    
saver_2 = tf.train.Saver()

matches = get_all_matches('training/')
lenmatches = len(matches)
print(lenmatches)

# Byrjum þjálfunina
print("start training")
validation_path = "validation/"
#print("starting error:" + str(avg_error(validation_path, sess)))
#print('%s: Step %d: Prediction accuracy = %.2f' % (datetime.now(), 0,
#                                                      prediction_accuracy()/float(60)))
iterations = 30000
eta        = 1e-4
prev_stop  = 0
probs      = 0.5
for i in range(prev_stop, prev_stop+iterations):
    #TODO: Skrifa þetta fall
    #input_batch, label_batch = prepare_train_batch(train_ids, batch_size, do_flips, do_rots, data_path)
    current_match      = matches[i]
    raw_match_movelist = current_match[8:]
    unpacked_movelist  = unpack('b'*60, raw_match_movelist)
    board              = initialize_game()
    player             = -1
    input_batch        = []
    label_batch        = []    

    
    for move in unpacked_movelist:
        if move == 0:
            break
        input_batch, label_batch = prep_and_append_training_batch(i, label_batch,
                                                                  move, input_batch, board, player)
        board = make_move(board, move, player)
        player = -1 if player is 1 else 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            player = -1 if player is 1 else 1
                
    train_step_1.run(session=sess_1, feed_dict={img_data_1:input_batch,
                                            ground_truths_1: label_batch,
                                            keep_prob_1:probs,
                                            learn_rate_1:eta})
    train_step_2.run(session=sess_2, feed_dict={img_data_2:input_batch,
                                            ground_truths_2: label_batch,
                                            keep_prob_2:probs,
                                            learn_rate_2:eta})
    #img_data_2, train_step_2, optimizer_2, ground_truths_2, loss_2, pred_up_2, keep_prob_2
    if (i % 50 is 0) and (i > 0 and i < 2000) or (i+1) == (iterations+prev_stop) or (i % 1000 is 0):  
        errors1, accs1 = policy_loss(validation_path, sess_1,
                                     img_data_1, ground_truths_1, keep_prob_1, loss_1, pred_up_1)
        errors2, accs2 = policy_loss(validation_path, sess_2,
                                     img_data_2, ground_truths_2, keep_prob_2, loss_2, pred_up_2)
        print('%s, Step %d, loss value 1 = %.3f, loss value 1 = %.3f' % \
                                     (datetime.now().strftime("%d. %b %H:%M:%S"), i, errors1, errors2))
        print('%s, Step %d, prediction accuracy 1 = %.3f, prediction accuracy 2  = %.3f' % \
                                     (datetime.now().strftime("%d. %b %H:%M:%S"), i, accs1, accs2))
        saver_1.save(sess_1, model_1)
        saver_2.save(sess_2, model_2)

print("done")


117298
start training
17. Oct 16:37:09, Step 0, loss value 1 = 39.936, loss value 1 = 5.486
17. Oct 16:37:09, Step 0, prediction accuracy 1 = 0.195, prediction accuracy 2  = 0.522


KeyboardInterrupt: 